In [ ]:
# Install dependencies for this example
# Note: This does not include itkwidgets, itself
import sys
!{sys.executable} -m pip install -U itk scikit-image

In [1]:
from urllib.request import urlretrieve
import os

import itk
import numpy as np
import skimage.io, skimage.filters, skimage.measure

from itkwidgets import view
import itkwidgets

In [2]:
# Download data
# Source: https://data.broadinstitute.org/bbbc/BBBC024/
file_name = 'HL50_cell_line_c00_03_extraction.tif'
if not os.path.exists(file_name):
    url = 'https://data.kitware.com/api/v1/file/5b61f16c8d777f06857c1949/download'
    urlretrieve(url, file_name)

In [3]:
image = itk.imread(file_name)

view(image, cmap=itkwidgets.cm.BrBG, annotations=False, vmax=800, ui_collapsed=True)

Viewer(annotations=False, cmap=['BrBG'], geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<…

In [10]:
LabelImageType = itk.Image[itk.UL, 3]
connected_components = itk.connected_component_image_filter(thresholded, ttype=(type(thresholded), LabelImageType))

In [8]:
# Segment the cells
smoothed = itk.smoothing_recursive_gaussian_image_filter(image)

thresholded = itk.moments_threshold_image_filter(smoothed,
                                                 inside_value=0,
                                                 outside_value=1)

connected_components = itk.connected_component_image_filter(thresholded)

In [11]:
view(label_image=connected_components)

TemplateTypeError: itk.ExtractImageFilter is not wrapped for input type `None`.

To limit the size of the package, only a limited number of
types are available in ITK Python. To print the supported
types, run the following command in your python environment:

    itk.ExtractImageFilter.GetTypes()

Possible solutions:
* If you are an application user:
** Convert your input image into a supported format (see below).
** Contact developer to report the issue.
* If you are an application developer, force input images to be
loaded in a supported pixel type.

    e.g.: instance = itk.ExtractImageFilter[itk.Image[itk.RGBPixel[itk.UC],2], itk.Image[itk.RGBPixel[itk.UC],2]].New(my_input)

* (Advanced) If you are an application developer, build ITK Python yourself and
turned to `ON` the corresponding CMake option to wrap the pixel type or image
dimension you need. When configuring ITK with CMake, you can set
`ITK_WRAP_${type}` (replace ${type} with appropriate pixel type such as
`double`). If you

In [ ]:
# We can combine the label map with the intensity image, add names for the labels, etc.
names = [(0, 'Background'), (1, 'First cell'), (2, 'Second cell')]
viewer = view(smoothed,
              label_image=connected_components,
              label_image_names=names,
              label_image_blend=0.8,
              rotate=True,
              gradient_opacity=0.5,
              slicing_planes=True)
viewer

Clicking on a label in a slice will toggle its weight.
We can also set label weights programmically.

In [ ]:
label_count = len(np.unique(connected_components))
label_weights = np.ones((label_count,), dtype=np.float32)
label_weights[1] = 0.1
label_weights[2] = 0.2
viewer.label_image_weights = label_weights

The images and labels can, of course, also come from *scikit-image*.

In [ ]:
image = skimage.io.imread(file_name)

view(image, cmap=itkwidgets.cm.BrBG, annotations=False, vmax=800, ui_collapsed=True)

In [ ]:
# Segment the cells
smoothed = skimage.filters.gaussian(image)

threshold = skimage.filters.threshold_isodata(smoothed)
thresholded = smoothed > threshold

connected_components = skimage.measure.label(thresholded)

In [ ]:
view(label_image=connected_components)